In [ ]:
# SETUP DEBUG FLEET
# (applies the first time step of actions in the example solution)

# MOVE THIS FILE TO THE BASE DIRECTORY IF YOU WANT TO USE IT
# or find a workaround for the error this throws in the greedy_profit dir
from system_state import SystemState
from utils import (load_problem_data,
                   load_solution)

solution = load_solution('./data/solution_example.json')
demand, datacenters, servers, selling_prices = load_problem_data()

state: SystemState = SystemState(datacenters=datacenters, servers=servers)

first_time_step_actions = solution.set_index('time_step').loc[1].to_dict('records')
state.update_state(first_time_step_actions)

print(state.fleet)

In [ ]:
# TESTING THE GREEDY PROFIT ALGORITHM/CALCULATING SERVERS TO BUY
from evaluation import get_actual_demand
from greedy_profit.helpers import get_unsatisfied_demand

actual_demand = get_actual_demand(demand)
unsatisfied_demand = get_unsatisfied_demand(actual_demand, state.fleet, 2)

for server_generation in unsatisfied_demand.index.unique():
    for latency_sensitivty in unsatisfied_demand.columns.unique():
        selling_price = selling_prices\
            .set_index(['server_generation', 'latency_sensitivity'])\
            .loc[server_generation, latency_sensitivty]

        d = unsatisfied_demand.loc[server_generation][latency_sensitivty]

        print(f"{server_generation}: {latency_sensitivty}: selling_price={selling_price}, demand={d}")
        print(f"Number of servers to buy = {d / selling_price}") # error

